# PROJEKT MODELU REGRESYJNEGO PRZEWIDYWANIA CEN MIESZKAŃ NA PODSTAWIE #

# ALGORYTMU LGBM #

  ## KWIECIEŃ 2025 ##

In [ ]:
import pandas as pd
import mlflow
from pycaret.regression import setup, pull, compare_models, plot_model, load_model
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import parallel_backend

In [ ]:
# tą komórkę uruchom jeżeli bazie bierzesz np. z DBeaver
# username = 'root'
# password = '1234'
# host = '127.0.0.1'
# port = 3306  
# database = 'projekt1'
# engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# df = pd.read_sql("SELECT * FROM saleflats", con=engine)

# engine.dispose()

In [ ]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Ceny'
MLFLOW_TAGS = {'data': 'Investoro_ceny', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df = pd.read_csv('sale_2024_14.csv', sep=',')

In [ ]:
df

In [ ]:
df.head(10)

In [ ]:
df.sample(10)

In [ ]:
df.info

In [ ]:
df[df.duplicated()]

In [ ]:
df.nunique()

In [ ]:
correlation_matrix = df[['Area', 'Price', 'BuiltYear', 'Floor', 'Floors', 'CommunityScore']].corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
df_beznull_price = df.dropna(subset=['Area'])

In [ ]:
df_beznull_price.isnull().sum()

In [ ]:
df_beznull_price

In [ ]:
Q1 = df_beznull_price["Price"].quantile(0.25)
Q3 = df_beznull_price["Price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price_p = df_beznull_price[~((df_beznull_price["Price"] < lower_bound) | (df_beznull_price["Price"] > upper_bound))]

In [ ]:
Q1 = df_price_p["PricePerSquareMeter"].quantile(0.25)
Q3 = df_price_p["PricePerSquareMeter"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price_a = df_price_p[~((df_price_p["PricePerSquareMeter"] < lower_bound) | (df_price_p["PricePerSquareMeter"] > upper_bound))]


In [ ]:
Q1 = df_price_a["Area"].quantile(0.25)
Q3 = df_price_a["Area"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_price1 = df_price_a[~((df_price_a["Area"] < lower_bound) | (df_price_a["Area"] > upper_bound))]

In [ ]:
df_beznull_price.describe().T

In [ ]:
df_price1.describe().T

In [ ]:
df_price1["Price"].value_counts(normalize=True)

In [ ]:
df_price1.sample(10)

In [ ]:
df_price1.nunique()

In [ ]:
unique_btype=df_price1['BuildingType'].unique()

unique_btype

In [ ]:
print(df_price1['Price'].nunique())

In [ ]:
print(df_price1['Price'].value_counts())

In [ ]:
df_price1.isnull().sum()

In [ ]:
df_price2 = df_price1.dropna(subset=['Price'])

In [ ]:
df_price2.isnull().sum()

In [ ]:
correlation_matrix = df_price2[['Area', 'Price', 'BuiltYear', 'Floor', 'Floors', 'CommunityScore']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

In [ ]:
sale_ids = df_price2['SaleId'].reset_index(drop=True)

In [ ]:
train_df = df_price2.sample(frac=0.9, random_state=42)
holdout_df = df_price2.drop(train_df.index)

In [ ]:
'''
exp = setup(
    data=df_price2,
    target='Price',
    session_id=1122,
    verbose=False,
    categorical_features=[
        'BuildingType',
        'BuildingCondition',
         ],
    keep_features=[
        'SaleId',
        'Location',
        'Area',
         ],
    ignore_features=[
        'RealPriceAfterRenovation',
        'OfferPrice',
        'OriginalPrice',
        'PricePerSquareMeter',
        'OriginalId',
        'PortalId',
        'Title',
        'Type',
        'OfferFrom',
        'TypeOfMarket',
        'OwnerType',
        'DateAddedToDatabase',
        'DateAdded',
        'DateLastModification',
        'DateLastRaises',
        'NewestDate',
        'AvailableFrom',
        'Link',
        'Phone',
        'MainImage',
        'OtherImages',
        'NumberOfDuplicates',
        'NumberOfRaises',
        'NumberOfModifications',
        'IsDuplicatePriceLower',
        'IsDuplicatePrivateOwner',
        'Score',
        'ScorePrecision',
        'CommunityScore',
        'NumberOfCommunityComments',
        'NumberOfCommunityOpinions',
        'Archive',
        'VoivodeshipNumber',
        'CountyNumber',
        'CommunityNumber',
        'RegionNumber',
        'KindNumber',
        'SubRegionNumber',
        'StreetNumber',
        'EncryptedId',
        ],
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartametowiec', 'Kamienica'],
                      'BuildingCondition': ['For_Renovation', 'Good', 'After_Renovation', 'Developer_State' ]},        
       
)
exp.dataset_transformed.sample(10)
'''

In [ ]:
exp = setup(
    data=df_price2,
    target='Price',
    verbose=False,
    session_id=1122,
    log_experiment=True,
    log_plots=True,
    experiment_name=MLFLOW_EXPERIMENT_NAME,
    experiment_custom_tags=MLFLOW_TAGS,
    categorical_features=[
        'BuildingType',
        'BuildingCondition',
         ],
    keep_features=[
        'SaleId',
        'Location',
        'Area',
         ],
    ignore_features=[
        'RealPriceAfterRenovation',
        'OfferPrice',
        'OriginalPrice',
        'PricePerSquareMeter',
        'OriginalId',
        'PortalId',
        'Title',
        'Type',
        'OfferFrom',
        'TypeOfMarket',
        'OwnerType',
        'DateAddedToDatabase',
        'DateAdded',
        'DateLastModification',
        'DateLastRaises',
        'NewestDate',
        'AvailableFrom',
        'Link',
        'Phone',
        'MainImage',
        'OtherImages',
        'NumberOfDuplicates',
        'NumberOfRaises',
        'NumberOfModifications',
        'IsDuplicatePriceLower',
        'IsDuplicatePrivateOwner',
        'Score',
        'ScorePrecision',
        'CommunityScore',
        'NumberOfCommunityComments',
        'NumberOfCommunityOpinions',
        'Archive',
        'VoivodeshipNumber',
        'CountyNumber',
        'CommunityNumber',
        'RegionNumber',
        'KindNumber',
        'SubRegionNumber',
        'StreetNumber',
        'EncryptedId',
        ],
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartametowiec', 'Kamienica'],
                      'BuildingCondition': ['For_Renovation', 'Good', 'After_Renovation', 'Developer_State' ]},
)
best_unbalanced_model = unbalanced_exp.compare_models()
unbalanced_metrics_df = pull()

In [ ]:
plt.figure(figsize=(12, 6))
plt.barh(unbalanced_metrics_df['Model'], unbalanced_metrics_df['R2'], color='skyblue')
plt.xlabel('R2')
plt.title('Porównanie dokładności modeli')
plt.gca().invert_yaxis()  # najlepszy model na górze
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
exp.plot_model(best_unbalanced_model, plot='error')

In [ ]:
exp.plot_model(best_unbalanced_model, plot='feature')

In [ ]:
tuned_model = exp.tune_model(best_unbalanced_model, n_iter=3, optimize='R2')

In [ ]:
best_model = exp.compare_models([best_unbalanced_model, tuned_model])

In [ ]:
exp.predict_model(best_unbalanced_model)

In [ ]:
exp.plot_model(best_model, plot='error')

In [ ]:
exp.plot_model(best_unbalanced_model, plot='feature')

In [ ]:
best_tuned_model = exp.tune_model(
    best_unbalanced_model,
    optimize="R2",
    choose_better=True,
    fold=5,
)

In [ ]:
exp.predict_model(best_tuned_model).head(10)

In [ ]:
exp.plot_model(best_tuned_model, plot='error')

In [ ]:
exp.plot_model(best_tuned_model, plot='feature')

In [ ]:
best_final_model = exp.finalize_model(best_unbalanced_model, experiment_custom_tags={"step": "final"})
best_final_model

In [ ]:
exp.save_model(best_final_model, '0-basic-model')

In [ ]:
exp.plot_model(best_final_model, plot='error')

In [ ]:
exp.predict_model(best_final_model, data=holdout_df)
predict_holdout_df = exp.pull()

In [ ]:
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
runs_df = mlflow.search_runs([mlflow_exp.experiment_id], order_by=["start_time DESC"])
run_id = runs_df['run_id'].values[0]
metrics = predict_holdout_df.drop(columns=["Model"]).to_dict(orient='records')[0]

with mlflow.start_run(run_id=run_id, nested=True):
    for key, value in metrics.items():
        mlflow.log_metric(key, value)

In [ ]:
predictions = exp.predict_model(best_final_model)

In [ ]:
predicted_column = predictions['prediction_label']
print(predicted_column)

In [ ]:
print(predictions[['prediction_label']].applymap('{:,.0f}'.format))

In [ ]:
df_last=predictions[['prediction_label','Price', ]].applymap('{:,.0f}'.format)


In [ ]:
df_last.to_csv('0_new_prices.csv')

In [ ]:
print(df_last.columns)

In [ ]:
print(df_last.head())

In [ ]:
df_last = df_last.reset_index()  
print(df_last.columns)  

In [ ]:
df_last.rename(columns={'index': 'SaleId'}, inplace=True)

In [ ]:
saleflats_df = pd.read_csv('sale_2024_14.csv')

In [ ]:
new_prices_df = pd.read_csv('0_new_prices.csv')

In [ ]:
print("saleflats_df.columns:", saleflats_df.columns.tolist())

In [ ]:
print("new_prices_df.columns:", new_prices_df.columns.tolist())

In [ ]:
new_prices_df = new_prices_df.rename(columns={'Unnamed: 0': 'SaleID', 'Price': 'NewPrice'})

In [ ]:
print("new_prices_df.columns:", new_prices_df.columns.tolist())

In [ ]:
print(new_prices_df.columns)

In [ ]:
print(saleflats_df.columns)

In [ ]:
merged_df = pd.merge(
    saleflats_df,
    new_prices_df[['SaleID', 'NewPrice']],
    left_on='SaleId',  # Kolumna w saleflats_df
    right_on='SaleID',  # Kolumna w new_prices_df
    how='left'
)

In [ ]:
merged_df

In [ ]:
# Krok 2: usuwanie dodatkowej kolumny klucza (opcjonalnie, bo mamy już 'SaleId')
merged_df.drop(columns=['SaleID'], inplace=True)

# Krok 3: przestawienie kolumny 'NewPrice' obok 'Price'
cols = merged_df.columns.tolist()

# znajdź indeks kolumny 'Price'
price_index = cols.index('Price')

# usuń NewPrice z listy i dodaj ją zaraz po Price
cols.remove('NewPrice')
cols.insert(price_index + 1, 'NewPrice')

# ustaw nową kolejność kolumn
merged_df = merged_df[cols]

In [ ]:
merged_df

In [ ]:
merged_df[merged_df.duplicated()]

In [ ]:
prediction_df = merged_df.copy()

In [ ]:
from pycaret.regression import predict_model

In [ ]:
prediction_df_clean = prediction_df.drop(columns=['Price'], errors='ignore')
predictions = predict_model(best_final_model, data=prediction_df_clean)

In [ ]:
predictions['RealPrice'] = prediction_df['Price']

In [ ]:
merged_df['PredictedPrice'] = predictions['prediction_label']

In [ ]:
predictions=predictions[['prediction_label',]].applymap('{:,.0f}'.format)

In [ ]:
# Przenieś kolumnę 'PredictedPrice' za 'NewPrice'
cols = list(merged_df.columns)
new_price_index = cols.index('NewPrice')
# Usuń z listy kolumn
cols.remove('PredictedPrice')
# Dodaj w odpowiednie miejsce
cols.insert(new_price_index + 1, 'PredictedPrice')
# Przekształć DataFrame
merged_df = merged_df[cols]

In [ ]:
merged_df.to_csv('0_new_prices_full.csv')

In [ ]:
merged_df = pd.read_csv('0_new_prices_full.csv')

In [ ]:
merged_df

In [ ]:
merged_df2=merged_df[['PredictedPrice',]].applymap('{:,.0f}'.format)

In [ ]:
merged_df2

In [ ]:
merged_df['PredictedPrice'] = merged_df2['PredictedPrice']

In [ ]:
merged_df.drop(columns=['NewPrice'], inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('uzupelnione_mieszkania_ceny.csv')

In [ ]:
# df_p = pd.read_csv('sale_2024_6.csv')

# 1. Kopia oryginalnej dużej tabeli
df = df_p.copy()

# 2. Dopinamy kolumnę z predykcjami po SaleId
if 'NewPrices' in merged_df.columns:
    df = df.merge(merged_df[['SaleId', 'NewPrices']], on='SaleId', how='left')
elif 'PredictedPrice' in merged_df.columns:
    df = df.merge(merged_df[['SaleId', 'PredictedPrice']].rename(columns={'PredictedPrice': 'NewPrices'}), on='SaleId', how='left')
else:
    raise ValueError("W merged_df nie znaleziono kolumny 'NewPrices' ani 'PredictedPrice'")

# 3. Tam, gdzie jest dostępna cena ('Price'), kopiujemy ją do 'NewPrices'
df.loc[df['Price'].notna(), 'NewPrices'] = df.loc[df['Price'].notna(), 'Price']

# 4. Przenosimy kolumnę 'NewPrices' za 'Price'
cols = df.columns.tolist()
if 'NewPrices' in cols:
    cols.remove('NewPrices')
    price_idx = cols.index('Price')
    cols.insert(price_idx + 1, 'NewPrices')
    df = df[cols]

# 5. Załaduj model z PyCaret
model = load_model('6_best_price_modelLGBM')

# 6. Przygotuj dane bez kolumny 'Price', by uniknąć błędu z NaN
df_for_prediction = df.drop(columns=['Price'], errors='ignore')

# 7. Przeprowadź predykcję
predicted_df = predict_model(model, data=df_for_prediction)

# 8. Powiąż wynik predykcji po SaleID
df = df.merge(predicted_df[['SaleID', 'prediction_label']], on='SaleID', how='left', suffixes=('', '_new'))

# 9. Nadpisz kolumnę 'NewPrices' nowymi przewidywaniami
df['NewPrices'] = df['prediction_label']

# 10. Przenosimy kolumnę 'NewPrices' za 'Price' (jeszcze raz)
cols = df.columns.tolist()
if 'NewPrices' in cols:
    cols.remove('NewPrices')
    price_idx = cols.index('Price')
    cols.insert(price_idx + 1, 'NewPrices')
    df = df[cols]

# 11. Zaktualizuj saleflats_b
saleflats_b = df

df2=df[['NewPrices',]].applymap('{:,.0f}'.format)

# Nadpisanie kolumny 'NewPrices' w df sformatowaną wersją
df['NewPrices'] = df2['NewPrices']

df

df.to_csv('6_new_prices_FULL2.csv')

df.sample(100)